In [7]:
import pandas as pd
import math
import random
import numpy as np

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics 

from sklearn.naive_bayes import GaussianNB

In [8]:
fruit = pd.read_table("C:/Users/Jerry Iyi-Ewuare/Downloads/fruit_data_with_colors.txt");
fruit.head()

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


In [9]:
X = fruit[["mass","width","height","color_score"]]
X.head()

,mass,width,height,color_score
0,192,8.4,7.3,0.55
1,180,8.0,6.8,0.59
2,176,7.4,7.2,0.60
3,86,6.2,4.7,0.80
4,84,6.0,4.6,0.79


In [10]:
y = fruit[["fruit_name"]]
y.head()

,fruit_name
0,apple
1,apple
2,apple
3,mandarin
4,mandarin


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4321)
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

In [12]:
for i in range(1,9):
    KNclassifier = KNeighborsClassifier(n_neighbors = i)  
    KNclassifier.fit(X_train, y_train.values.ravel())
    y_prediction = KNclassifier.predict(X_test)
    print(confusion_matrix(y_test, y_prediction))
    print("accuracy score:", metrics.accuracy_score(y_test, y_prediction))

[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [0 0 0 6]]
accuracy score: 1.0
[[4 0 0 3]
 [0 2 0 0]
 [0 3 0 0]
 [0 0 0 6]]
accuracy score: 0.6666666666666666
[[4 0 0 3]
 [0 2 0 0]
 [0 3 0 0]
 [0 1 0 5]]
accuracy score: 0.6111111111111112
[[2 0 0 5]
 [0 2 0 0]
 [0 2 0 1]
 [0 0 0 6]]
accuracy score: 0.5555555555555556
[[2 0 0 5]
 [0 2 0 0]
 [0 2 0 1]
 [0 2 0 4]]
accuracy score: 0.4444444444444444


In [13]:
Gclassifier = GaussianNB()  
Gclassifier.fit(X_train, y_train.values.ravel())
y_prediction = Gclassifier.predict(X_test)
print(confusion_matrix(y_test, y_prediction))
print("accuracy score:", metrics.accuracy_score(y_test, y_prediction))

[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [2 0 0 4]]
accuracy score: 0.8888888888888888


In [220]:
def uniqueSet(dataset):
    result = set(dataset)
    return result

In [221]:
def createDict(dataset):
    result = dict()
    for i in dataset:
        result[i] = []
    return result

In [222]:
def createClass(diction, dataset):
    newset = set()
    while len(dataset) > 0:
        newset.add(dataset.pop())
    for i in diction:
        name = newset.pop()
        diction[i] = fruit(name)
    return diction

In [1]:
def adddatatoclass(X, y, dictionary):
    keys = list(dictionary.keys())
    types = list(y)
    for k in keys:
        for i in range (len(types)):
            if k == types[i]:
               dictionary[k].add(X[i]) 

In [224]:
def createarrayofmeans(classdict):
    keys = list(classdict.keys())
    model = {}
    meanarray = []
    for k in keys:
        model[k] = []
    for k in keys:
        for i in range(4):
            meanarray.append(classdict[k].mean(i))
        model[k] = meanarray.copy()
        meanarray.clear()
    return model

In [225]:
def createarrayofstdev(classdict):
    keys = list(classdict.keys())
    model = {}
    meanarray = []
    for k in keys:
        model[k] = []
    for k in keys:
        for i in range(4):
            average = classdict[k].mean(i)
            meanarray.append(classdict[k].stdev(i, average))
        model[k] = meanarray.copy()
        meanarray.clear()
    return model

In [226]:
def calpdf(rows, modelmean, modelstdev, classdict):
    totalsize = 0
    keys = list(modelmean.keys())
    model = {}
    for k in keys:
        model[k] = 1
        totalsize = totalsize + classdict[k].size
    for k in keys:
        totalprob = 1
        for i in range(4):
            exponent = math.exp(-(math.pow(rows[i] - modelmean[k][i], 2)/(2 * math.pow(modelstdev[k][i], 2))))
            pdf = (1/(math.sqrt(2 * math.pi) * modelstdev[k][i]))*exponent
            totalprob *= pdf
        totalprob *= (classdict[k].size/totalsize)
        model[k] = totalprob
    return model

In [227]:
def predict(modelmean, modelstdev, classdict, rows):
    probabilities = calpdf(rows, modelmean, modelstdev, classdict)
    bestLabel, bestProb = None, -1
    for labels in probabilities:
        if bestLabel is None or probabilities[labels] > bestProb:
            bestProb = probabilities[labels]
            bestLabel = labels
    return bestLabel

In [228]:
def getPredictions(modelmean, modelstdev, classdict, X):
    predictions = []
    for rows in X:
        result = predict(modelmean, modelstdev, classdict, rows)
        predictions.append(result)
    return predictions

In [229]:
class fruit:
    def __init__(self, label):
        self.label = label
        self.data = []
        self.size = 0
        
    def add(self, newdata):
        self.data.append(newdata)
        self.size = self.size + 1
    
    def mean(self, index):
        total = 0
        for i in range(self.size):
            total = total + self.data[i][index]
        return total/(len(self.data))
        
    def stdev(self, index, average):
        total = 0
        for i in range(self.size):
            total = total + pow(self.data[i][index] - average, 2)
        variance = (total)/float(len(self.data)-1)
        return math.sqrt(variance)

In [230]:
result = uniqueSet(y_train['fruit_name'])
diction = createDict(result)
classdict = createClass(diction, result)
adddatatoclass(X_train, y_train['fruit_name'], classdict)
modelmean = createarrayofmeans(classdict)
modelstdev = createarrayofstdev(classdict)
predictions = getPredictions(modelmean, modelstdev, classdict, X_test)
print(confusion_matrix(y_test, predictions))
print("accuracy score:", metrics.accuracy_score(y_test, predictions))

[[7 0 0 0]
 [0 2 0 0]
 [0 0 3 0]
 [2 0 0 4]]
accuracy score: 0.8888888888888888
